In [1]:
import qiime2
import skbio
from qiime2.plugins import diversity
import pandas as pd

This cell is markdown so that it can't be run again by accident, since that would overwrite the Artifacts that we're using in the paper (and the PCoA is non-deterministic). 

```
md = qiime2.Metadata.load('../sample-metadata-temp.tsv')

pc1s = []

metrics = ['unweighted_unifrac', 'weighted_unifrac', 'bray_curtis', 'jaccard']
for metric in metrics:
    dm = qiime2.Artifact.load('../100_otus/cm900/%s_distance_matrix.qza' % metric)
    dm = qiime2.plugins.diversity.actions.filter_distance_matrix(dm, md, where="IncludedIn2017Analysis='Yes'").filtered_distance_matrix
    dm.save('%s-dm-802-samples.qza' % metric)
    pc = qiime2.plugins.diversity.actions.pcoa(dm).pcoa
    pc.save('%s-pc-802-samples.qza' % metric)
    pc1s.append(pc.view(skbio.stats.ordination.OrdinationResults).samples[0])                                                                 
```           

In [15]:
faith = qiime2.Artifact.load('../100_otus/cm900/faith_pd_vector.qza').view(pd.Series)
obs_otus = qiime2.Artifact.load('../100_otus/cm900/observed_otus_vector.qza').view(pd.Series)
df = pd.DataFrame(data=pc1s + [faith, obs_otus], index=['OTU100-%s-PC1' % m for m in metrics + ['faith_pd', 'observed_otus']]).T.dropna()
df.to_csv('diversity-values.tsv', sep='\t')

In [16]:
df.shape

(802, 6)

In [17]:
df.corr()

,OTU100-unweighted_unifrac-PC1,OTU100-weighted_unifrac-PC1,OTU100-bray_curtis-PC1,OTU100-jaccard-PC1,OTU100-faith_pd-PC1,OTU100-observed_otus-PC1
OTU100-unweighted_unifrac-PC1,1.000000,0.229811,0.610533,0.798971,0.816179,0.719729
OTU100-weighted_unifrac-PC1,0.229811,1.000000,0.759551,0.373302,-0.004245,-0.078872
OTU100-bray_curtis-PC1,0.610533,0.759551,1.000000,0.744785,0.326891,0.274133
OTU100-jaccard-PC1,0.798971,0.373302,0.744785,1.000000,0.506845,0.486456
OTU100-faith_pd-PC1,0.816179,-0.004245,0.326891,0.506845,1.000000,0.922474
OTU100-observed_otus-PC1,0.719729,-0.078872,0.274133,0.486456,0.922474,1.000000
